In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import os
import shutil

import matplotlib.pyplot as plt
import numpy as np
import time
import pandas as pd
import json
import matplotlib.image as mpimg

import seaborn as sb
import torch
from torch import nn
from torch import optim

from torchvision import datasets, transforms, models
from collections import OrderedDict

import h5py

from PIL import Image


In [2]:
train_filenames = os.listdir('train')
train_cat = os.listdir('train2\\cat')
train_dog = os.listdir('train2\\dog')
#如需对比去除异常值的的结果，可以参照日期更早的csv文件，或者解除注释本行以下部分并注释以上部分，运行本文件和main.ipynb
# train_filenames = os.listdir('train')
# train_cat = list(filter(lambda x:x[:3] == 'cat', train_filenames))
# train_dog = list(filter(lambda x:x[:3] == 'dog', train_filenames))

# def rmrf_mkdir(dirname):
#     if os.path.exists(dirname):
#         shutil.rmtree(dirname)
#     os.mkdir(dirname)


# rmrf_mkdir('train2')
# os.mkdir('train2\\cat')
# os.mkdir('train2\\dog')


# for filename in train_cat:
#     os.symlink('E:\\AI\\MLND_CN_P7_DogvsCat\\train\\'+filename, 'train2\\cat\\'+filename)

# for filename in train_dog:
#     os.symlink('E:\\AI\\MLND_CN_P7_DogvsCat\\train\\'+filename, 'train2\\dog\\'+filename)

    
# test_filenames = os.listdir('test')
# test_filenames_no_jpg = list([x[:-4] for x in test_filenames])


# rmrf_mkdir('test2')
# for filename_no_jpg, filename in zip(test_filenames_no_jpg, test_filenames):
#     rmrf_mkdir('test2\\' + filename_no_jpg)

#     os.symlink('E:\\AI\\MLND_CN_P7_DogvsCat\\test\\'+filename, 'test2\\' + filename_no_jpg + '\\' + filename)

In [3]:
train_dir = 'E:\\AI\\MLND_CN_P7_DogvsCat\\train2'
test_dir = 'E:\\AI\\MLND_CN_P7_DogvsCat\\test2'

In [4]:
def load_data(size=224,shuffle=False,batch=16):
    train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                           transforms.Resize(int(size*1.39)),
                                           transforms.RandomResizedCrop(size),
                                           transforms.RandomHorizontalFlip(),
                                           transforms.ToTensor(),
                                           transforms.Normalize([0.485, 0.456, 0.406], 
                                                                [0.229, 0.224, 0.225])])
    test_transforms = transforms.Compose([transforms.Resize(int(size*1.39)),
                                           transforms.CenterCrop(size),
                                           transforms.ToTensor(),
                                           transforms.Normalize([0.485, 0.456, 0.406], 
                                                                [0.229, 0.224, 0.225])])

    train_datasets = datasets.ImageFolder(train_dir, transform=train_transforms)

    test_datasets = datasets.ImageFolder(test_dir, transform=test_transforms)
    # TODO: 使用datasets定义dataloaders
    train_loader = torch.utils.data.DataLoader(train_datasets, batch_size=batch, shuffle=shuffle)

    test_loader = torch.utils.data.DataLoader(test_datasets, batch_size=batch)
    return train_datasets, test_datasets, train_loader, test_loader

In [5]:
train_datasets, test_datasets, train_loader, test_loader = load_data()

In [6]:
model_vgg = models.vgg19_bn(pretrained=True)
model_vgg.classifier = nn.Linear(model_vgg.classifier[0].in_features, 2048)
model_vgg.cuda()
for p in model_vgg.parameters():
    p.requires_grad = False

model_res = models.resnet152(pretrained=True)
model_res.fc = nn.Linear(model_res.fc.in_features, 2048)
model_res.cuda()
for p in model_res.parameters():
    p.requires_grad = False

model_den = models.densenet161(pretrained=True)
model_den.classifier = nn.Linear(model_den.classifier.in_features, 2048)
model_den.cuda()
for p in model_den.parameters():
    p.requires_grad = False

In [7]:
def bottle_save(model_s):
    def get_bottle(model_s, loader_s):
        bottle_neck_list = []
        for data, labels in loader_s:
            model_s.eval()
            with torch.no_grad():
                data = data.cuda()
                bottle = model_s(data)
                bottle_neck_list += bottle.data.cpu().numpy().tolist()
        return bottle_neck_list


    train = get_bottle(model_s, train_loader)
    test = get_bottle(model_s, test_loader)
    train_classes = []
    for i in train_filenames:
        if i in train_cat:
            train_classes.append(0)
        elif i in train_dog:
            train_classes.append(1)
    with h5py.File("gap_%s.h5"%str(model_s)[:3]) as h:
            h.create_dataset("train", data=train)
            h.create_dataset("test", data=test)
            h.create_dataset("train_label", data=train_classes)

以下过程请谨慎，可能持续30分钟左右，消耗大量cpu性能和部分GPU占用

In [8]:
a = time.time()
bottle_save(model_res)
bottle_save(model_vgg)
bottle_save(model_den)
a = time.time() - a
print(a)

1466.8500611782074
